In [36]:
import os
import pandas as pd
import sqlite3
import sys

def load_data(query):
    '''
    Retrieve data for year/location/group from the internet
    and return data (or write data to file, if `outpath` is
    not `None`).
    '''
    path = os.path.join( os.path.dirname(os.getcwd()), "src/data/battery_use-000025.db")
    
    conn = sqlite3.connect(path)
    
    
    return pd.read_sql_query(query, conn)

In [37]:
query = '''select INPUT_DESCRIPTION, MEASUREMENT_TIME, VALUE
from inputs i 
join 
(
	SELECT *
	FROM counters_ull_time_data
	
	UNION
	
	SELECT *
	FROM counters_ll_time_data
) all_data
WHERE 
i.id_input = all_data.id_input
ORDER BY MEASUREMENT_TIME'''

In [38]:
information = load_data(query)

## Data Preprocessing Part
def show_original(x):
    return x
table = pd.pivot_table(information, values='VALUE', columns=['INPUT_DESCRIPTION'],
                    index=['MEASUREMENT_TIME'], aggfunc=show_original)
table['Battery Tag'] = table['Battery Tag'].fillna(method = 'ffill')
table

INPUT_DESCRIPTION,Battery Charging.,Battery Current Capacity (mWh),Battery Cycle Count,Battery Designed Capacity (mWh),Battery Discharging.,Battery Flag,Battery Full Charged Capacity (mWh),Battery Left (%).,Battery Life Time (Seconds).,Battery MaxCapacity (mWh).,Battery Rate (mW),Battery Tag,Battery Voltage (mv),Battery estimatedTime (Seconds).
MEASUREMENT_TIME,,,,,,,,,,,,,,
2020-11-20 07:20:53.332,NaN,NaN,12.0,70000.0,NaN,NaN,69680.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN
2020-11-20 07:20:53.399,1.0,68380.0,NaN,NaN,0.0,9.0,NaN,98.0,4.294967e+09,69680.0,5803.0,4.0,17375.0,4.294967e+09
2020-11-20 07:21:23.331,NaN,NaN,12.0,70000.0,NaN,NaN,69680.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN
2020-11-20 07:21:23.396,1.0,68420.0,NaN,NaN,0.0,9.0,NaN,98.0,4.294967e+09,69680.0,5699.0,4.0,17375.0,4.294967e+09
2020-11-20 07:21:53.330,NaN,NaN,12.0,70000.0,NaN,NaN,69680.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-20 09:19:23.303,0.0,51190.0,NaN,NaN,1.0,1.0,NaN,73.0,1.894600e+04,69720.0,-9583.0,5.0,16026.0,1.894600e+04
2020-11-20 09:19:53.235,NaN,NaN,12.0,70000.0,NaN,NaN,69720.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2020-11-20 09:19:53.302,0.0,51100.0,NaN,NaN,1.0,1.0,NaN,73.0,1.894600e+04,69720.0,-10374.0,5.0,16035.0,1.894600e+04


In [34]:
## Data Exploration Part
print('----------Battery Tag 4------------------')
display(table.loc[table['Battery Tag'] == 4].nunique())
print('----------Battery Tag 5------------------')

table.loc[table['Battery Tag'] == 5].nunique()


----------Battery Tag 4------------------


INPUT_DESCRIPTION
Battery Charging.                       2
Battery Current Capacity (mWh)         21
Battery Cycle Count                     1
Battery Designed Capacity (mWh)         1
Battery Discharging.                    2
Battery Flag                            2
Battery Full Charged Capacity (mWh)     1
Battery Left (%).                       3
Battery Life Time (Seconds).            1
Battery MaxCapacity (mWh).              1
Battery Rate (mW)                      22
Battery Tag                             1
Battery Voltage (mv)                   21
Battery estimatedTime (Seconds).        1
dtype: int64

----------Battery Tag 5------------------


INPUT_DESCRIPTION
Battery Charging.                        1
Battery Current Capacity (mWh)         177
Battery Cycle Count                      1
Battery Designed Capacity (mWh)          1
Battery Discharging.                     1
Battery Flag                             1
Battery Full Charged Capacity (mWh)      1
Battery Left (%).                       28
Battery Life Time (Seconds).            30
Battery MaxCapacity (mWh).               1
Battery Rate (mW)                      171
Battery Tag                              1
Battery Voltage (mv)                   165
Battery estimatedTime (Seconds).        30
dtype: int64

In [35]:
(4956 - table.loc[table['Battery Tag'] == 5].isna().sum().sum()) / 3024 ## more than 85% of our data is tagged 5
table = table.loc[table['Battery Tag'] == 5]
table

INPUT_DESCRIPTION,Battery Charging.,Battery Current Capacity (mWh),Battery Cycle Count,Battery Designed Capacity (mWh),Battery Discharging.,Battery Flag,Battery Full Charged Capacity (mWh),Battery Left (%).,Battery Life Time (Seconds).,Battery MaxCapacity (mWh).,Battery Rate (mW),Battery Tag,Battery Voltage (mv),Battery estimatedTime (Seconds).
MEASUREMENT_TIME,,,,,,,,,,,,,,
2020-11-20 07:41:23.314,NaN,NaN,12.0,70000.0,NaN,NaN,69720.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2020-11-20 07:41:23.381,0.0,69630.0,NaN,NaN,1.0,1.0,NaN,100.0,4.294967e+09,69720.0,-10438.0,5.0,17112.0,4.294967e+09
2020-11-20 07:41:53.313,NaN,NaN,12.0,70000.0,NaN,NaN,69720.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2020-11-20 07:41:53.381,0.0,69530.0,NaN,NaN,1.0,1.0,NaN,100.0,2.209900e+04,69720.0,-12171.0,5.0,17095.0,2.209900e+04
2020-11-20 07:42:23.313,NaN,NaN,12.0,70000.0,NaN,NaN,69720.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-20 09:19:23.303,0.0,51190.0,NaN,NaN,1.0,1.0,NaN,73.0,1.894600e+04,69720.0,-9583.0,5.0,16026.0,1.894600e+04
2020-11-20 09:19:53.235,NaN,NaN,12.0,70000.0,NaN,NaN,69720.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2020-11-20 09:19:53.302,0.0,51100.0,NaN,NaN,1.0,1.0,NaN,73.0,1.894600e+04,69720.0,-10374.0,5.0,16035.0,1.894600e+04
